<a href="https://colab.research.google.com/github/iluxa0517/ZnuText/blob/main/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install Whisper and FFmpeg
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

from google.colab import drive
import os
from IPython.display import HTML, display, Javascript
from google.colab import output

# Mount Google Drive
drive.mount('/content/drive')

# Initialize paths
audio_files_path = ""
output_path = ""

# Function to list directories and generate HTML for folder selection
def generate_folder_ui(path="/content/drive/MyDrive"):
    if os.path.exists(path):
        folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]

        if folders:
            folder_buttons_html = ''
            for idx, folder in enumerate(folders, 1):
                folder_buttons_html += f"""
                <button class="folder-button" onclick="selectFolder('{folder}')">{folder}</button>
                """

            ui_html = f"""
            <div style="text-align: center;">
                <h3>Select a Folder from Google Drive</h3>
                <div>{folder_buttons_html}</div>
                <p id="selectedFolder" style="font-size: 16px; color: #555;">Please select a folder.</p>
                <p id="pathDisplay" style="font-size: 16px; color: #555; font-weight: bold;"></p>
            </div>
            <style>
                .folder-button {{
                    margin: 5px;
                    padding: 10px 20px;
                    font-size: 16px;
                    background-color: #4CAF50;
                    color: white;
                    border: none;
                    border-radius: 5px;
                    cursor: pointer;
                    transition: background-color 0.3s;
                }}
                .folder-button:hover {{
                    background-color: #45a049;
                }}
            </style>
            <script>
                function selectFolder(folderName) {{
                    // Notify Python callback
                    google.colab.kernel.invokeFunction('process_selected_folder', [folderName], {{}});

                    // Display folder selection info
                    document.getElementById("selectedFolder").innerText = "Selected folder: " + folderName;
                    document.getElementById("selectedFolder").style.color = "#28a745";
                    document.getElementById("pathDisplay").innerText = "Path: /content/drive/MyDrive/" + folderName;

                    // Show Start Processing button
                    document.getElementById("startButton").style.display = "inline-block";
                }}
            </script>
            """
            display(HTML(ui_html))
        else:
            print("No folders found in the specified path.")
    else:
        print(f"Path {path} does not exist.")

# Python function to process the selected folder and update audio_files_path and output_path
def process_selected_folder(folder_name):
    global audio_files_path, output_path
    audio_files_path = f"/content/drive/MyDrive/{folder_name}"
    output_path = os.path.join(audio_files_path, "Output")
    os.makedirs(output_path, exist_ok=True)  # Ensure output folder is created

    # Print both paths for verification
    print(f"Updated audio_files_path: {audio_files_path}")
    print(f"Updated output_path: {output_path}")

# Register the callback function for JavaScript
output.register_callback('process_selected_folder', process_selected_folder)

# Function to list all audio files in the folder and print them
def list_audio_files():
    audio_files = [f for f in os.listdir(audio_files_path) if f.endswith(('.mp3', '.wav', '.m4a', '.ogg'))]
    if audio_files:
        files_html = "<h3>Identified Files:</h3><ul>"
        for audio_file in audio_files:
            files_html += f"<li>{audio_file}</li>"
        files_html += "</ul>"
        display(HTML(files_html))
    else:
        display(HTML("<p>No audio files found to process.</p>"))

    return audio_files

# Function to process the audio files and display full output
def process_audio_files():
    audio_files = list_audio_files()
    if not audio_files:
        return

    # Loop through and process each file
    for audio_file in audio_files:
        full_path_to_file = os.path.join(audio_files_path, audio_file)
        output_file_name = audio_file.rsplit('.', 1)[0] + ".txt"
        output_file_path = os.path.join(output_path, output_file_name)

        # Check if the transcription for this file already exists
        if os.path.exists(output_file_path):
            display(HTML(f"<p>Skipping already processed file: {audio_file}</p>"))
            continue  # Skip the rest of the loop and move to the next file

        # Construct the Whisper command
        command = f"whisper '{full_path_to_file}' --model medium --language uk"
        display(HTML(f"<p>Executing command: {command}</p>"))

        # Execute the Whisper command using "!" (Colab's shell command execution)
        result = !{command}

        # Display the output of the Whisper command in real-time
        display(HTML(f"<pre>{result}</pre>"))

        # Write the output (transcription) to the desired file
        with open(output_file_path, 'w') as file:
            file.write("\n".join(result))

    display(HTML("<p>Processing completed.</p>"))

# HTML for the button to start processing
def display_ui():
    ui_html = """
    <div style="text-align: center;">
        <h3>Start Processing Audio Files</h3>
        <button id="startButton" onclick="processFiles()"
                style="padding: 10px 20px; font-size: 16px; background-color: #4CAF50; color: white; border: none; border-radius: 5px; cursor: pointer; display: none;">
            Start Processing
        </button>
    </div>
    <script>
        function processFiles() {
            google.colab.kernel.invokeFunction('process_audio_files', [], {});
        }
    </script>
    """
    display(HTML(ui_html))

# Register the Python function for processing files with JavaScript callback
output.register_callback('process_audio_files', process_audio_files)

# Generate the folder selection UI and display the processing button
generate_folder_ui()
display_ui()


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-_c7me6o1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-_c7me6o1
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803669 sha256=bec2465edfe6ecc01db48f8d55085cfb972fd2c60b7be83a5773a59f794be21d
  Stored in directory: /tmp/pip-ephem-wheel-cache-q2951l8b/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204

Updated audio_files_path: /content/drive/MyDrive/Test transcript
Updated output_path: /content/drive/MyDrive/Test transcript/Output


100%|█████████████████████████████████████| 1.42G/1.42G [00:22<00:00, 67.7MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper